# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Import the dataset with: 

```sh
mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json
```

In [54]:
# Import dependencies
from pymongo import MongoClient, errors
from pprint import pprint as pp
from bson.objectid import ObjectId

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [7]:
# list databases
databases = mongo.list_database_names()

databases

['admin', 'classDB', 'config', 'local', 'travel_db', 'uk_food']

In [16]:
# confirm that our new database was created
if 'uk_food' in databases: print('database exists')

database exists


In [17]:
# assign the uk_food database to a variable name
uk_food = mongo['uk_food']

In [19]:
# review the collections in our new database
uk_food.list_collection_names()

['establishments']

In [24]:
# assign the collection to a variable
establishments = uk_food['establishments']

In [23]:
# review a document in the establishments collection
pp(establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 1034540,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64de0f6bcb13883c3c7d4992'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1034540',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [26]:
# Create a dictionary for the new restaurant data
new_restaurant_dict = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [44]:
# Insert the new restaurant into the collection
try:
    r = establishments.insert_one(new_restaurant_dict)
    print('Inserted with ID: ', r.inserted_id)
except errors.DuplicateKeyError:
    print('record already exists')


record already exists


In [49]:
# Check that the new restaurant was inserted
result = establishments.find({"BusinessName":"Penang Flavours"})

for r in result: pp(r)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64de16ba055ed102e9c1560b'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [51]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

ids = establishments.distinct("BusinessTypeID", {"BusinessType": "Restaurant/Cafe/Canteen"})

for id in ids: print(id)

1



3. Update the new restaurant with the `BusinessTypeID` you found.

In [57]:
# Update the new restaurant with the correct BusinessTypeID
penang_id = ObjectId('64de16ba055ed102e9c1560b')

result = establishments.update_one(
    {"_id": penang_id},
    {"$set": {"BusinessTypeID": 1}}
)

print(result.modified_count, 'records updated')


1 records updated


In [61]:
# Confirm that the new restaurant was updated
result = establishments.find_one({"_id": penang_id})
pp(result['BusinessTypeID'])

1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [63]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({"LocalAuthorityName": "Dover"})


994

In [64]:
# Delete all documents where LocalAuthorityName is "Dover"
result = establishments.delete_many({"LocalAuthorityName": "Dover"})

print(result.deleted_count, 'deleted')


994 deleted


In [65]:
# Check if any remaining documents include Dover
establishments.count_documents({"LocalAuthorityName": "Dover"})


0

In [66]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('64de0f6bcb13883c3c7d4c76'),
 'FHRSID': 289352,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000002460',
 'BusinessName': 'The Ship',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': '23 The Stade',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6AB',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2015-08-27T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.188537', 'latitude': '51.08084'},
 'RightToReply': '',
 'Distance': 4591.912144538092,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [76]:
# Find documents with string latitudes and longitudes

affected_records_query = {
    "$or": [
        {"geocode.latitude": {"$type": "string"}},
        {"geocode.longitude": {"$type": "string"}}
    ]
}

str_documents = establishments.find(affected_records_query)

print(establishments.count_documents(affected_records_query), 'affected records')


38786 affected records


In [77]:
# Change the data type from String to Decimal for latitude/longitude

for doc in str_documents:
    lat = float(doc['geocode']['latitude'])
    lon = float(doc['geocode']['longitude'])
    
    establishments.update_one(
        {'_id': doc['_id']},
        {
            '$set': {
                "geocode.latitude": lat,
                "geocode.longitude": lon
            }
        }
    )


In [78]:
# Check that the coordinates are now numbers
print(establishments.count_documents(affected_records_query), 'affected records')


0 affected records


In [87]:
# Find documents with string ratings

affected_records_query = {
    "$or": [
        {"RatingValue": {"$type": "string"}}
    ]
}

str_documents = establishments.find(affected_records_query)

print(establishments.count_documents(affected_records_query), 'affected records')

38728 affected records


In [89]:
# Change the data type from String to Decimal for RatingValue

for doc in str_documents:
    
    try:
        rating = int(doc['RatingValue'])
    except ValueError:
        pass

    establishments.update_one(
        {'_id': doc['_id']},
        {
            '$set': {
                "RatingValue": rating
            }
        }
    )


In [92]:
# remaining records with a string RatingValue

print(establishments.count_documents(affected_records_query), 'affected records')

1 affected records


In [93]:
establishments.find_one(affected_records_query)

{'_id': ObjectId('64de0f6bcb13883c3c7d4ca4'),
 'FHRSID': 1534966,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000080027',
 'BusinessName': 'Putters!',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Folkestone Harbour',
 'AddressLine2': 'Harbour Approach Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': '',
 'Phone': '',
 'RatingValue': 'AwaitingInspection',
 'RatingKey': 'fhrs_awaitinginspection_en-gb',
 'RatingDate': '1901-01-01T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': None,
  'Structural': None,
  'ConfidenceInManagement': None},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.18590330311705, 'latitude': 51.0783519967076},
 'RightToReply': '',
 'Distance': 4591.914705329384,
 'NewRatingPending': False,
 'met